In [2]:
import sddk
import pandas as pd
pd.options.display.max_columns = 1000 # to see all columns
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')
import nltk

In [3]:
# find local version of the input dataset
local_paths = !find ~/Projects -name "LIREg.parquet"
print(local_paths)

['/Users/kasev/Projects/LIRE_ETL/data/large_data/LIREg.parquet']


In [4]:
# read LIRE dataset from zenodo or locally, if already available
try:
    LIRE = gpd.read_parquet(local_paths[0])
    print("local copy found and loaded")
except:# !!! use carefully - takes time
    q = input("local file not found, try another put, or input 'y' for download")
    if q == "y":
        LIRE = gpd.read_file("https://zenodo.org/record/5074774/files/LIREg.geojson?download=1", driver="geoJSON")
        print("file downloaded from zenodo")
        LIRE.to_parquet("../data/large_data/LIREg.parquet")

local copy found and loaded


In [8]:
# records with valid EDH-ID
edh_ids_list = LIRE[LIRE["EDH-ID"].notnull()]["EDH-ID"].tolist()
len(edh_ids_list)

53823

In [9]:
# records with valid & UNIQUE EDH-ID
edh_ids_unique = LIRE[LIRE["EDH-ID"].notnull()]["EDH-ID"].unique().tolist()
len(edh_ids_unique)

52708

In [10]:
# EDH-IDs appearing at least twice:
dup_ids = [id_tup[0] for id_tup in nltk.FreqDist(edh_ids_list).most_common() if id_tup[1] > 1]
len(dup_ids)

946

In [11]:
# dataframe of duplicate rows (i.e. every second instance of the same id
LIRE[LIRE["EDH-ID"].notnull()].loc[LIRE.duplicated(subset="EDH-ID"), :]

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
164,EDCS-12700138,"ILGR 00158 = AE 1928, 00015 = AE 1937, 00114 =...",Achaia,Achaia,Nikolara / Nikopoli / Nicopolis,"['Nikolara', 'Nikopoli', 'Nicopolis']",-24,,"['Augusti/Augustae', 'tituli operum', 'tituli ...","['tituli operum', 'tituli sacri']","['Augusti/Augustae', 'viri']",{ },,,[Nep]tuno [et Ma]rt[i Imp(erator) Caesa]r div[...,[Nep]tuno [et Ma]rt[i Imp(erator) Caesa]r div[...,http://db.edcs.eu/epigr/partner.php?s_language...,-24.0,-24,Augusti/Augustae; tituli operum; tituli sacri;...,39.009084,20.734332,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD023503,None,Neptuno et Marti Imperator Caesar divi Iuli fi...,Cowey,votive inscription,None,"K.A. Romaios, AD 9, 1924/25, 1-4; eik. 1. - AE...",checked with photo,None,[ ]TVNO [ ]RT[ ]R DIV[ ] F VICT[ ]RIT[ ]O QVOD...,"[{'nomen': 'Caesar+', 'praenomen': 'Imp.+', 'p...",None,None,block,Epirus,None,[Nep]tuno [et Ma]rt[i Imp(erator) Caesa]r Div[...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,"Nicopolis, bei",2013-07-23,Ípeiros,Nikópolis,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Mehrere Blöcke des Siegesdenkmals von Actium....,https://www.trismegistos.org/text/120977,http://www.flickr.com/photos/dandiffendale/835...,['https://edh-www.adw.uni-heidelberg.de/fotos/...,120977,[],Neptuno et Marti Imperator Caesar Divi Iuli fi...,29 BC,21,unbestimmt,189.0,Block,138,unbestimmt,1000.0,80,Weihinschrift,votive inscription,Certain,NaN,NaN,NaN,NULL,block,Certain,Epirus,Certain,Greece,Certain,Nicopolis,Estimated,Ípeiros,Certain,Nikópolis,Certain,"Nikópolis, Augustusmonument",Certain,29 BC,tuno rtr Div f victrito quod pro blic gest in ...,Neptuno et Marti Imperator Caesar Divi Iuli fi...,"Nikópolis, Augustusmonument",None,None,names of pagan deities,None,data available,None,NaN,Latin,-29.0,"[20.7355, 39.0276]",Neptuno et Marti Imperator Caesar Divi Iuli fi...,big,False,Nicopolis (Achaea),65,41221.0,"[20.733395, 39.026505]",big,0.002373,votive inscription,1.0,POINT (20.73550 39.02760)
204,EDCS-50300300,JARCE-1994-161,Aegyptus,Aegyptus,Abu Sha|ar,Abu Sha|ar,{ },,{ },{ },{ },{ },,,]VM[1]LI[,]VM[1]LI[,None,NaN,None,None,27.368929,33.682834,None,None,None,HD052304,None,VM LI,Platz,None,None,"AE 2004, 1641b.",provisional,None,]ANIS LIMITIBVS APTA IN LITO[ ] / CVRANTE AVRE...,{ },None

In [23]:
# remove the duplicates
# (using pandas deduplicated() method)
LIRE = LIRE.loc[~(LIRE.duplicated(subset="EDH-ID")) | (LIRE["EDH-ID"].isnull())]
len(LIRE)

136190

In [29]:
# login to sciencedata.dk
s = sddk.cloudSession("sciencedata.dk", "SDAM_root/SDAM_data/LIRE", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/SDAM_data/LIRE/


In [33]:
# upload data to sciencedata.dk
s.write_file("LIRE_v1-1.geojson", LIRE)

Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/LIRE/LIRE_v1-1.geojson"


In [32]:
# save the file locally
# a) as geojson:
LIRE.to_file("../data/large_data/LIRE_v1-1.geojson", driver="GeoJSON")

In [34]:
# b) as parquet:
LIRE.to_parquet("../data/large_data/LIRE_v1-1.parquet")